In [1]:
import json
import math
# from graphviz import Graph
import node as Node

# load the data
with open('UK_cities.json') as f:
    data = json.load(f)    
# print(data)

#visualise the data
# u = Graph('cities', filename='cities.gv',
#             node_attr={'color': 'lightblue2', 'style': 'filled'})
# u.attr(size='6,6')

# for k0,k1 in data.items():
#     for k,v in k1.items():
#         u.edge(k0,k,label=str(v['weight']))    
# u.view()

In [4]:
def preprocess(data,environment_aware_factor,speed):
    # preprocessing step 1
    city_list = []
    city_weight_map_list = []
    environment_aware = 0
    # creating a map of all nodes to all nodes where path exists.

    for k0,k1 in data.items():
        for k,v in k1.items():
            city_list.append(k0)
            city_list.append(k)
            city_weight_map_list.append([k0,k,is_environment_aware(environment_aware_factor,float(v['weight']),speed)])

    city_list = list(set(city_list))
    # preprocessing step 2
    # generating a dict of the above map
    city_map_dict = {}
    for i in city_list:
        alist = []
        for j in city_weight_map_list:
            if i in j:
                alist.append(j)
                city_map_dict[i] = alist
    return city_map_dict

In [5]:
def generate_graph(city_map_dict):
    graph = {}
    # generate a graph of the above map 
    for k,v in city_map_dict.items():
        _reachable = []
        for i in v:
            _reachable.append(Node.Node(i[0] if i[0] != k else i[1], 0, i[2], None))
        graph[k] = _reachable
    return graph
    

In [2]:
def is_environment_aware(environment_aware_factor,distance,speed):
    '''
    if the environment_aware_factor is 0, return 
        cost as the distance
        if environment_aware_factor is 1 calculat-
        e the cost as the sum of the overall driv-
        ing time plus the overall cost of the air 
        pollution due to your driving
        if environment_aware_factor is -1 calculat-
        e the cost as overall cost is the sum of t-
        he car rental fee plus the total (likely) 
        fines
    params, environment_aware_factor, integer, (0,
        1,-1)
    return, cost
    '''
    if environment_aware_factor == 0:
        cost = distance
    elif environment_aware_factor == 1:
        cost = (distance/speed)+(0.00001*speed*speed)*(distance/speed)
    elif environment_aware_factor == -1:
        speed_limit = distance # assumption as mentioned in question
        # fine_factor is a value between 0 and 1
        fine_factor = 0 if speed < speed_limit else 1 - math.exp(-(speed-speed_limit))
        # if fine_factor > 0 put 1000 fine, else 0
        fine =  1000 * (1 if fine_factor > 0 else 0)
        car_rental = (distance/speed)*100
        cost = car_rental + fine
    else:
        cost = 0
    return cost

In [3]:
import gc

def ucf(graph,start,destination):
    
    def recursive_expand_nodes(graph,frontier,visited_nodes,destination,previous_frontier):
        if len(frontier) is not 0: # if empty return failure
            node = frontier[0] # assign and pop the lowest cost node from front
            frontier.remove(node)
            if node.return_state() not in visited_nodes: # if the current node is not visited then
                previous_frontier = [node.return_state()] if previous_frontier is None else visited_nodes
                print("frontier = ",[(node.return_state(),str(node.return_cost()),previous_frontier)])
                child_node_list = graph[node.return_state()] # get all the possible path from the current node as children
                # if current_node is not equal to destination node, expand its children
                if node.return_state() not in [destination]: 
                    # for each child in parent_node set cost, depth and add the child to frontier for it to be explored 
                    print('Frontier selected & Child nodes of',node.return_state(),'are')
                    child_list = []
                    for child_node in child_node_list:                        
                        child_node.put_cost(node.return_cost() + child_node.return_cost())
                        child_node.put_depth(node.return_depth()+1)
#                         print("increasing depth to",child_node.return_depth(), child_node.return_state())
                        child_list.append(['depth '+str(child_node.return_depth()),child_node.return_state(),'total cost incured',child_node.return_cost()])
                        child_node.put_node(node)
                        frontier.append(child_node)
                    v = [(i.return_state(),i.return_depth(),i.return_cost()) for i in frontier]
                    print("Exploring all frontiers = ")
                    for i in v:
                        print(str([(node.return_state(),str(node.return_cost())),i]))
                    frontier.sort(key=lambda nodes: nodes.cost) # sort frontier according to lowest cost
                    print("Sorting frontier based on cost  = ",[(node.return_state(),str(node.return_cost()),[(i.return_state(),i.return_depth(),i.return_cost()) for i in frontier])])
                    
                visited_nodes.append(node.return_state()) # add the current node to visited node
                #print(frontier)
                print("explored =",set(visited_nodes))
                print("destination =",destination)
                print("*"*25)
                
                # if current node is destination node then return with the path and cost
                if node.return_state() in [destination]:
                    previous_nodes = []
                    previous_nodes.append(node)
                    while node.return_node() is not None:
                        node = node.return_node()
                        previous_nodes.append(node)

                    previous_nodes = previous_nodes[::-1] # reversing the order

                    count = 0
                    print("Path =")
                    for child in previous_nodes:                        
                        if len(previous_nodes)-1 > count:
                            count = count + 1
                            print(child.return_state(),"->",previous_nodes[count].return_state(),"," ,str(previous_nodes[count].return_cost() - child.return_cost())," units")

                        # if london to london
                        if len(previous_nodes) == 1:
                            print(child.return_state(),"->",child.return_state(),",",str(previous_nodes[-1].return_cost()),"units")
                    print("Total Cost =",str(previous_nodes[-1].return_cost())," units")
                    del frontier    
                    return "Completed trace"               
            recursive_expand_nodes(graph,frontier,visited_nodes,destination,previous_frontier)
        else:
            print("Distance: Infinity")
            print("Route: None")
        
    frontier = []
    visited_nodes = []
    frontier.append(Node.Node(start))
    recursive_expand_nodes(graph,frontier,visited_nodes,destination,None)

    gc.collect()

In [6]:
# normal ucs - q1 b
environment_aware_factor = 0
speed = 0
city_map_dict = preprocess(data,environment_aware_factor,speed)
print(city_map_dict['london'])
graph = generate_graph(city_map_dict)
print(graph['london'][0].return_state())
ucf(graph,'london','aberdeen')

[['birmingham', 'london', 110.0], ['brighton', 'london', 52.0], ['bristol', 'london', 116.0], ['cambridge', 'london', 54.0], ['cardiff', 'london', 161.0], ['carlisle', 'london', 302.0], ['dover', 'london', 71.0], ['exeter', 'london', 172.0], ['glasgow', 'london', 396.0], ['hull', 'london', 172.0], ['leeds', 'london', 198.0], ['liverpool', 'london', 198.0], ['oxford', 'london', 57.0]]
birmingham
frontier =  [('london', '0.0', ['london'])]
Frontier selected & Child nodes of london are
Exploring all frontiers = 
[('london', '0.0'), ('birmingham', 1, 110.0)]
[('london', '0.0'), ('brighton', 1, 52.0)]
[('london', '0.0'), ('bristol', 1, 116.0)]
[('london', '0.0'), ('cambridge', 1, 54.0)]
[('london', '0.0'), ('cardiff', 1, 161.0)]
[('london', '0.0'), ('carlisle', 1, 302.0)]
[('london', '0.0'), ('dover', 1, 71.0)]
[('london', '0.0'), ('exeter', 1, 172.0)]
[('london', '0.0'), ('glasgow', 1, 396.0)]
[('london', '0.0'), ('hull', 1, 172.0)]
[('london', '0.0'), ('leeds', 1, 198.0)]
[('london', '0.0

[('glasgow', '396.0'), ('glasgow', 2, 415.0)]
[('glasgow', '396.0'), ('dover', 2, 418.0)]
[('glasgow', '396.0'), ('cambridge', 3, 418.0)]
[('glasgow', '396.0'), ('portsmouth', 3, 421.0)]
[('glasgow', '396.0'), ('manchester', 2, 422.0)]
[('glasgow', '396.0'), ('swansea', 2, 427.0)]
[('glasgow', '396.0'), ('leeds', 3, 431.0)]
[('glasgow', '396.0'), ('glasgow', 4, 431.0)]
[('glasgow', '396.0'), ('glasgow', 4, 431.0)]
[('glasgow', '396.0'), ('newcastle', 3, 432.0)]
[('glasgow', '396.0'), ('carlisle', 3, 432.0)]
[('glasgow', '396.0'), ('cardiff', 3, 435.0)]
[('glasgow', '396.0'), ('glasgow', 3, 436.0)]
[('glasgow', '396.0'), ('carlisle', 3, 437.0)]
[('glasgow', '396.0'), ('exeter', 2, 437.0)]
[('glasgow', '396.0'), ('exeter', 3, 437.0)]
[('glasgow', '396.0'), ('brighton', 3, 438.0)]
[('glasgow', '396.0'), ('nottingham', 4, 438.0)]
[('glasgow', '396.0'), ('leeds', 2, 443.0)]
[('glasgow', '396.0'), ('carlisle', 2, 444.0)]
[('glasgow', '396.0'), ('liverpool', 4, 445.0)]
[('glasgow', '396.0'), 

In [7]:
# q2 c
environment_aware_factor = 1
speed = 316.22
city_map_dict = preprocess(data,environment_aware_factor,speed)
print(city_map_dict['london'])
graph = generate_graph(city_map_dict)
print(graph['london'][0].return_state())
ucf(graph,'london','aberdeen')

[['birmingham', 'london', 0.6957010854468408], ['brighton', 'london', 0.32887687675668836], ['bristol', 'london', 0.7336484173803048], ['cambridge', 'london', 0.3415259874011764], ['cardiff', 'london', 1.0182534068812852], ['carlisle', 'london', 1.9100157073176902], ['dover', 'london', 0.4490434278793245], ['exeter', 'london', 1.0878235154259692], ['glasgow', 'london', 2.504523907608627], ['hull', 'london', 1.0878235154259692], ['leeds', 'london', 1.2522619538043136], ['liverpool', 'london', 1.2522619538043136], ['oxford', 'london', 0.36049965336790846]]
birmingham
frontier =  [('london', '0.0', ['london'])]
Frontier selected & Child nodes of london are
Exploring all frontiers = 
[('london', '0.0'), ('birmingham', 1, 0.6957010854468408)]
[('london', '0.0'), ('brighton', 1, 0.32887687675668836)]
[('london', '0.0'), ('bristol', 1, 0.7336484173803048)]
[('london', '0.0'), ('cambridge', 1, 0.3415259874011764)]
[('london', '0.0'), ('cardiff', 1, 1.0182534068812852)]
[('london', '0.0'), ('ca

[('penzance', '1.9416384839289105'), ('carlisle', 3, 2.0238577031180824)]
[('penzance', '1.9416384839289105'), ('london', 3, 2.0301822584403264)]
[('penzance', '1.9416384839289105'), ('carlisle', 2, 2.0301822584403264)]
[('penzance', '1.9416384839289105'), ('manchester', 3, 2.0365068137625704)]
[('penzance', '1.9416384839289105'), ('hull', 2, 2.0365068137625704)]
[('penzance', '1.9416384839289105'), ('aberystwyth', 3, 2.0491559244070583)]
[('penzance', '1.9416384839289105'), ('york', 2, 2.0681295903737906)]
[('penzance', '1.9416384839289105'), ('birmingham', 3, 2.0681295903737906)]
[('penzance', '1.9416384839289105'), ('swansea', 3, 2.074454145696034)]
[('penzance', '1.9416384839289105'), ('birmingham', 3, 2.074454145696034)]
[('penzance', '1.9416384839289105'), ('penzance', 2, 2.0807787010182786)]
[('penzance', '1.9416384839289105'), ('newcastle', 3, 2.0871032563405225)]
[('penzance', '1.9416384839289105'), ('portsmouth', 3, 2.0934278116627665)]
[('penzance', '1.9416384839289105'), ('

In [8]:
# q1 d
environment_aware_factor = -1
speed = 316.22
city_map_dict = preprocess(data,environment_aware_factor,speed)
print(city_map_dict['london'])
graph = generate_graph(city_map_dict)
print(graph['london'][0].return_state())
ucf(graph,'london','aberdeen')

[['birmingham', 'london', 1034.7859085446842], ['brighton', 'london', 1016.4442476756689], ['bristol', 'london', 1036.6833217380304], ['cambridge', 'london', 1017.0767187401176], ['cardiff', 'london', 1050.9139206881284], ['carlisle', 'london', 1095.503130731769], ['dover', 'london', 1022.4527227879324], ['exeter', 'london', 1054.3925115425968], ['glasgow', 'london', 125.22927076086268], ['hull', 'london', 1054.3925115425968], ['leeds', 'london', 1062.6146353804313], ['liverpool', 'london', 1062.6146353804313], ['oxford', 'london', 1018.0254253367908]]
birmingham
frontier =  [('london', '0.0', ['london'])]
Frontier selected & Child nodes of london are
Exploring all frontiers = 
[('london', '0.0'), ('birmingham', 1, 1034.7859085446842)]
[('london', '0.0'), ('brighton', 1, 1016.4442476756689)]
[('london', '0.0'), ('bristol', 1, 1036.6833217380304)]
[('london', '0.0'), ('cambridge', 1, 1017.0767187401176)]
[('london', '0.0'), ('cardiff', 1, 1050.9139206881284)]
[('london', '0.0'), ('carli

[('sheffield', '638.1633040288406'), ('manchester', 6, 1650.1802542533678)]
[('sheffield', '638.1633040288406'), ('newcastle', 6, 1677.0602744924417)]
[('sheffield', '638.1633040288406'), ('nottingham', 6, 1650.1802542533678)]
[('sheffield', '638.1633040288406'), ('oxford', 6, 1681.1713364113589)]
[('sheffield', '638.1633040288406'), ('penzance', 6, 748.213269242932)]
[('sheffield', '638.1633040288406'), ('portsmouth', 6, 1705.8377079248621)]
[('sheffield', '638.1633040288406'), ('swansea', 6, 1700.1454683448233)]
Sorting frontier based on cost  =  [('sheffield', '638.1633040288406', [('aberdeen', 5, 639.4282461577382), ('york', 5, 653.9750806400606), ('aberdeen', 5, 658.7186136234266), ('newcastle', 5, 676.1115678957686), ('penzance', 6, 731.7690215672633), ('aberdeen', 5, 737.7774966795268), ('aberdeen', 6, 745.050913920688), ('penzance', 6, 745.6833849851369), ('penzance', 6, 748.213269242932), ('brighton', 1, 1016.4442476756689), ('cambridge', 1, 1017.0767187401176), ('oxford', 1, 